### Step 0. Download SpaCy and language model
If you don't already have the SpaCy library installed, the code below downloads it along with the English language model. This notebook should work for any [language with a SpaCy model](https://spacy.io/models). Just substitute the name of the model for *en_core_web_sm* (For instance, if you wanted to use Lithuanian, you can replace `en_core_web_sm` with `lt_core_news_sm`). Places where you need to do this substitution are commented in the code.

In [ ]:
#Replace es_core_news_lg with another model name here for other languages
!python -m spacy download es_core_news_lg

### Step 1. Import modules
To use a [SpaCy language model](https://spacy.io/models) other than English, replace `en_core_web_sm` with the model name in the cell below.

In [ ]:
# os is used for navigating directories
import os
# spacy is used for identifying the subjects and verbs
import spacy
#Replace en_core_web_sm with another model name here for other languages
import es_core_news_lg
#Replace en_core_web_sm with another model name here for other languages
nlp = spacy.load("es_core_news_lg")

## Exploring spaCy tagging

In [ ]:
example = nlp("Empezó Maximiliano sus estudios el 69, y su hermano y su tía le ponderaban lo bonita que era la Farmacia y lo mucho que con ella se ganaba, por ser muy caros los medicamentos y muy baratas las primeras materias: agua del pozo, ceniza del fogón, tierra de los tiestos, etcétera... El pobre chico, que era muy dócil, con todo se mostraba conforme.")

for token in example:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
from spacy import displacy
displacy.render(example, style="dep")

### Step 3. Find subjects
The code cells below reads each text file in the directory, finds every subject and verb, and writes it to a CSV file along with the filename it's from. The files are processed alphabetically, so that if something breaks (e.g. text files bigger than 1 MB may trigger a memory error), you can figure out what files are left to be done.

By default, the CSV file is called `charverbs.csv` and gets created inside the directory with the text files. You can give the file a different name in the code cell below, but keeping `.csv` is recommended.

Doing the NLP parse can be time-consuming, particularly if you have a large number of files. If your text is on the scale of hundreds of novels, expect it to take hours.

In [ ]:
charverbfile = 'fortunata-jacinda-verbs.csv'

In [ ]:
#Opens the output file
with open(charverbfile, 'w') as out:
    out.write('Filename, Subject, Verb' + '\n')
    #Sorts the files alphabetically
    for filename in sorted(os.listdir('.')):
        #Looks for .txt files
        if filename.endswith('.txt'):
            #Opens each file
            with open(filename, 'r') as bookfile:
                #Reads in the text in the file
                book = bookfile.read()
                #NLP parse of the text
                doc = nlp(book)
                #Noun chunks are the part of the SpaCy dependency parse that we need
                for chunk in doc.noun_chunks:
                    #If the dependency relation is 'nsubj' (noun subject)
                    if chunk.root.dep_ == 'nsubj':
                        #Write the filename, the noun chunk, the verb, and then a newline character
                        strsubj = str(chunk.text)
                        cleansubj = strsubj.replace(',', '')                        
                        out.write(filename + ', ' + cleansubj + ', ' + chunk.root.head.text + '\n')
                        print(filename + ', ' + cleansubj + ', ' + chunk.root.head.text + '\n')